# Re-parameterising 2d Gaussian Profiles

In the general case of a multivariate Gaussian, the PDF is as follows:

$$
f_{\mathbf X}(x_1,\ldots,x_k) = \frac{\exp\left(-\frac 1 2 ({\mathbf x}-{\boldsymbol\mu})^\mathrm{T}{\boldsymbol\Sigma}^{-1}({\mathbf x}-{\boldsymbol\mu})\right)}{\sqrt{(2\pi)^k|\boldsymbol\Sigma|}}
$$

For the 2d case, setting

$$
\boldsymbol\mu = \begin{pmatrix} \mu_X \\ \mu_Y \end{pmatrix}, \quad
    \boldsymbol\Sigma = \begin{pmatrix} \sigma_X^2 & \rho \sigma_X \sigma_Y \\
                             \rho \sigma_X \sigma_Y  & \sigma_Y^2 \end{pmatrix}
$$

so that 
$$
    |\boldsymbol\Sigma| = \sigma_X^2 \sigma_Y^2(1 -  \rho^2)
$$

and 

$$
\boldsymbol\Sigma^{-1} = \frac{1}{|\boldsymbol\Sigma|} \begin{pmatrix} 
                                \sigma_Y^2 & -\rho \sigma_X \sigma_Y \\
                             -\rho \sigma_X \sigma_Y  & \sigma_X^2 
                             \end{pmatrix}
$$

Then substituting and simplifying gives [Eq 1]:
$$
\begin{align}
    f(x,y) &=
      \frac{1}{2 \pi  \sigma_X \sigma_Y \sqrt{1-\rho^2}}
      \exp\left(
        -\frac{1}{2(1-\rho^2)}\left[
          \frac{(x-\mu_X)^2}{\sigma_X^2} +
          \frac{(y-\mu_Y)^2}{\sigma_Y^2} -
          \frac{2\rho(x-\mu_X)(y-\mu_Y)}{\sigma_X \sigma_Y}
        \right]
      \right)\\ 
\end{align}
$$

For a source profile, the normalising constant factor is simply replaced by $A$, the peak amplitude.
If there is no correlation ($\rho =0$, which can alternatively be viewed as the major/minor profile axes being aligned with x/y) then this simplifies to the 'separable' 2D-Gaussian:
$$
\begin{align}
    f(x,y) &=
      A
      \exp\left(
        -\frac{1}{2}\left[
          \frac{(x-\mu_X)^2}{\sigma_X^2} +
          \frac{(y-\mu_Y)^2}{\sigma_Y^2}
        \right]
      \right)\\
\end{align}
$$

Alternatively we can use the following parameterisation, where $\theta$ represents the rotation of the major-axis of the Gaussian *counterclockwise* relative to the x-positive axis, and $\sigma_{maj}$, $\sigma_{min}$ represent the equivalent standard deviations in x and y if the Gaussian were *not rotated* - (i.e. the semimajor and semiminor axes) [Eq 2]:

$$
f(x,y) = A \exp\left(- \left(a(x - \mu_X)^2 + b(x-\mu_X)(y-\mu_Y) + c(y-\mu_Y)^2 \right)\right)
$$

Where
$$
a = \frac{1}{2}\left(\frac{\cos^2\theta}{\sigma_{maj}^2} + \frac{\sin^2\theta}{\sigma_{min}^2}\right) \\
b = \frac{1}{2}\left(\frac{\sin2\theta}{\sigma_{maj}^2} + \frac{\sin2\theta}{\sigma_{min}^2}\right) \\
c = \frac{1}{2}\left(\frac{\sin^2\theta}{\sigma_{maj}^2} + \frac{\cos^2\theta}{\sigma_{min}^2}\right) \\
$$

(This is the opposite rotation to that shown on Wikipedia, and matches the Astropy implementation.)

This is convenient when representing source profiles - we can immediately read off the relative size of the semi-major and semi-minor axes, interpret the source rotation angle, etc. Note, $\sigma_{maj}$ as defined here is the standard deviation equivalent length - to get the FWHM of the source profile you must multiply this by a factor of $2\sqrt{2\ln{2}}$ (see e.g. [Mathworld](http://mathworld.wolfram.com/GaussianFunction.html) for a derivation). (This matches the implementations in Astropy and S-Extractor, but PySE quotes the HWHM, i.e. larger by a factor $\sqrt{2\ln{2}}$.)

However, the rotation-angle parameterisation makes comparison of source fits trickier. For $\mu$ and $\sigma$ values we can use standard 'are these floating point values close' routines, but $\theta$ presents problems. For a start, even if we confine it to the range $(-\pi,\pi)$, we can still get two approximate fits with reported rotation angles just inside each of those bounds which actually represent very nearly identical fits. Second, when $\sigma_x=\sigma_y$, the rotation angle $\theta$ is effectively degenerate (since the profile is circular), so we can ignore it in comparisons. How do we smoothly transition between ignoring it completely and taking it into account for elongated profiles, when comparing noisy fits?

It seems a better approach would be to compare covariance matrices in the reference x-y frame. That way we effectively replace $\theta$ with the correlation $\rho$, which varies smoothly in the range $(-1,1)$, and simply tends to zero as a the Gaussian circularizes.

To do so, we take advantage of the fact that the covariance matrix for the rotated Gaussian is simply:

$$
\Sigma_{rot} = \begin{pmatrix} \sigma_{maj}^2 & 0 \\
                             0  & \sigma_{min}^2 \end{pmatrix}
$$

in the rotated frame. To transform to the reference frame we define the rotation matrix:

$$
R = \begin{pmatrix} cos(\theta) & sin(\theta) \\
                             -sin(\theta)  & \cos(\theta) \end{pmatrix}
$$

and apply a change of basis transform:

$$
\Sigma_{ref} = R\,\Sigma_{rot}\,R^{-1}
$$

Then we compare reference-frame covariance matrices (or specifically, correlation coeffcients) for the source fits in question.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fastimgproto.sourcefind.fit import Gaussian2dFit
import numpy as np
import astropy.units as u

In [ ]:
from astropy.coordinates import Angle, SkyCoord

In [ ]:
g1 = Gaussian2dFit(x_centre=0,
                   y_centre=0,
                   amplitude=1,
                   semimajor=1.5,
                   semiminor=1,
                   theta=np.pi/4,
                  )

g1

In [ ]:
g1.covariance

In [ ]:
g1.correlation

In [ ]:
g1.theta = -np.pi/4
g1.correlation

In [ ]:
g1.theta = np.pi/4
g1.semimajor = 1000
g1.correlation

In [ ]:
g1.semimajor = 1.0001
g1.correlation